In [3]:
# VENDO COM IMAGENS IMPORTADAS

import cv2
import supervision as sv
from ultralytics import YOLO
from flask import Flask, request, render_template, send_from_directory, Response
import threading
import os
import sqlite3

app = Flask(__name__)

# pasta para onde vão os uploads
UPLOAD_FOLDER = 'uploads'

# pasta para onde vão os resultados
RESULTS_FOLDER = 'results'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULTS_FOLDER, exist_ok=True)

cap = None 
frame = None

# caminho do analisador
path = 'C:\\Users\\ct67ca\\Desktop\\Cana\\runs\\train4\\weights\\best.pt'
model = YOLO(path) # modelo YOLO

# rota para abrir a página html
# @app.route('/')
# def upload():
#     return render_template('pagina.html') # pagina principal

def init_db():
    connection = sqlite3.connect('imagens.db')
    cursor = connection.cursor()

    cursor.execute(''' 
    CREATE TABLE IF NOT EXISTS imagens(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        caminho TEXT NOT NULL     
    )
''')
    
    connection.commit()
    connection.close()

# rota de upload das imagens
@app.route('/upload', methods=['POST'])
def upload_image():
    # erro se o arquivo não for encontrado no request
    if 'file' not in request.files:
        return 'Nenhum arquivo encontrado', 400
    
    # erro se nenhum arquivo for selecionado (sem nome)
    file = request.files['file']
    if file.filename == '':
        return 'Nenhum arquivo selecionado', 400

    # Se não cair em nenhuma condicional, a imagem é salva e processada
    image_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(image_path)
    
    # erro se não conseguir abrir a imagem (frame)
    frame = cv2.imread(image_path)
    if frame is None:
        return 'Erro ao abrir a imagem', 400
    
    # função que anota os quadros
    box_annotator = sv.BoxAnnotator()

    # função que anota os labels
    label_annotator = sv.LabelAnnotator()

    image = model(frame)[0]

    # de fato realizando as detecções das imagens com o modelo YOLO
    detections = sv.Detections.from_ultralytics(image)

    # labels que aparecem no print
    labels = [
        f"{class_name}{confidence: .2f}"
        for class_name, confidence in zip(detections['class_name'], detections.confidence)
    ]

    # de fato anotando de acordo com as detecções
    annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # salvando a imagem processada
    result_path = os.path.join(RESULTS_FOLDER, file.filename)
    cv2.imwrite(result_path, annotated_frame)

    connection = sqlite3.connect('imagens.db')
    cursor = connection.cursor()
    cursor.execute('INSERT INTO imagens(nome, caminho) VALUES (?,?)', (file.filename, result_path))

    return render_template('pagina_de_resultados.html', filename=file.filename)

@app.route('/results/<filename>')
def get_result(filename):
    return send_from_directory(RESULTS_FOLDER, filename)

@app.route('/camera')
def open_camera():
    global cap
    cap = cv2.VideoCapture(0)
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

def generate_frames():
    global cap
    if not cap.isOpened():
        print("Erro ao abrir a câmera")
        return

     # função que anota os quadros
    box_annotator = sv.BoxAnnotator()

    # função que anota os labels
    label_annotator = sv.LabelAnnotator()

    while True:
        success, frame = cap.read()
        if not success:
            break

        # detecção de fato com a câmera aberta
        results = model(frame)[0]
        detections = sv.Detections.from_ultralytics(results)

        labels = [
            f"{class_name}{confidence:.2f}"
            for class_name, confidence in zip(detections['class_name'], detections.confidence)
        ]

        annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
        annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

        ret, buffer = cv2.imencode('.jpg', annotated_frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/stop_camera')
def stop_camera():
    global cap
    if cap is not None:
        cap.release()
    return "Câmera parada."

if __name__ == '__main__':
    app.run(debug=True, port=5001)





 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1